In [ ]:
#Create structure for consort data information
consort = { 
    #initial data information
    "total_rows_loaded": None,
    "total_admissions": None,

    #inclusion/exclusion criteria
    "total_nippv_6h": None,
    "total_fio2_60": None,
    "total_pco2_45": None,
    "total_ph_7.35": None,

    #cohort size and failures BEFORE dropping missing data
    "patients_pre_missing": None,
    "failures_pre_missing": None,
    "imv_fail_pre_missing": None,
    "death_fail_pre_missing": None,
    "both_fail_pre_missing": None,

    #cohort size and failures AFTER dropping missing data
    "patients_post_missing": None,
    "failures_post_missing": None,
    "imv_fail_post_missing": None,
    "death_fail_post_missing": None,
    "both_fail_post_missing": None,
}

In [ ]:
import pandas as pd
import os

# import dataset
df = pd.read_parquet('../output/study_cohort_NIPPV_&_ICU.parquet')

print(f'Total rows loaded: {len(df)}')
consort["total_rows_loaded"] = len(df)

admissions = df['hospitalization_id'].nunique()
print(f'Total ICU NIPPV Admissions: {admissions}')
consort["total_admissions"] = admissions

In [ ]:
# Convert to standard pandas datetime
df['event_time'] = pd.to_datetime(df['event_time'], utc = True)
df['admission_dttm'] = pd.to_datetime(df['admission_dttm'], utc = True)

# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

In [ ]:
# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

vital_sign_cols = ['heart_rate', 'respiratory_rate', 'spo2', 'sbp', 'temp_c']

# Identify First Vital Sign
vitals = df[df[vital_sign_cols].notna().any(axis=1)]
vitals = vitals.sort_values(['hospitalization_id', 'event_time'])

first_vital = (
    vitals.groupby('hospitalization_id')
          .first()
          .reset_index()[['hospitalization_id', 'event_time']]
          .rename(columns={'event_time': 'first_vital_time'})
)

# Identify First NIPPV
nippv = df[df['device_category'] == 'NIPPV']
nippv = nippv.sort_values(['hospitalization_id', 'event_time'])

first_nippv = (
    nippv.groupby('hospitalization_id')
         .first()
         .reset_index()[['hospitalization_id', 'event_time']]
         .rename(columns={'event_time': 'first_nippv_time'})
)

# Merge First NIPPV with First Vital and Calculate Time differene
merged = first_vital.merge(first_nippv, on='hospitalization_id')
merged['time_to_NIPPV'] = (
    merged['first_nippv_time'] - merged['first_vital_time']
).dt.total_seconds() / 3600

# Filter dataframe to contain only hospitalizations where NIPPV occured within 6 hours of first vital sign
eligible_ids = merged[merged['time_to_NIPPV'] <= 6]['hospitalization_id']
df = df[df['hospitalization_id'].isin(eligible_ids)]

# Merge time_to_NIPPV and first_vital_time back into main dataframe
df = df.merge(
    merged[['hospitalization_id', 'time_to_NIPPV']],
    on='hospitalization_id',
    how='left'
)
df = df.merge(
    first_vital[['hospitalization_id', 'first_vital_time']],
    on='hospitalization_id',
    how='left'
)

# Print total rows and admissions
print(f'Total rows loaded: {len(df)}')
admissions = df['hospitalization_id'].nunique()
print(f'Total NIPPV < 6 hrs: {admissions}')
consort["total_nippv_6h"] = admissions

In [ ]:
# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

#Calculate nippv_start_time
df['nippv_start_time'] = (
    df['first_vital_time'] + pd.to_timedelta(df['time_to_NIPPV'], unit='h')
)

#Calculate time_since_nippv
df['time_since_nippv'] = (
    df['event_time'] - df['nippv_start_time']
).dt.total_seconds() / 3600

#Filter to rows w/in 48 hours of NIPPV start
df = df[df['time_since_nippv'] <= 48]

# Print total rows and admissions
print(f'Total rows loaded: {len(df)}')
admissions = df['hospitalization_id'].nunique()
print(f'Total NIPPV < 6 hrs: {admissions}')

In [ ]:
# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

#Filter to 1 hour after NIPPV initiation
df_PreNIPPV = df[df['event_time'] < df['nippv_start_time'] + pd.Timedelta(hours=1)]

# Get the maximum fio2_set for each hospitalization_id prior to NIPPV initiation
max_fio2_PreNIPPV = df_PreNIPPV.groupby('hospitalization_id')['fio2_set'].max().reset_index()

# Identify eligible hospitalizations where max fio2_set <= .6 prior to NIPPV initiation
eligible_fio2 = max_fio2_PreNIPPV[max_fio2_PreNIPPV['fio2_set'] <= .6]['hospitalization_id']

# Filter the df to keep only eligible fio2 hospitalizations
df = df[df['hospitalization_id'].isin(eligible_fio2)].reset_index(drop=True)

#Print total admissions
admissions = df['hospitalization_id'].nunique()
print(f'Total NIPPV < 6 hrs & fio2_set <= 60: {admissions}')
consort["total_fio2_60"] = admissions

In [ ]:
# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

#Update df_PreNIPPV with only filtered admissions
df_PreNIPPV = df[df['event_time'] < df['nippv_start_time'] + pd.Timedelta(hours=1)]

# Get minimum pco2_arterial and pco2_venous prior to NIPPV initiation for each hospitalization_id
min_pco2_arterial = df_PreNIPPV.groupby('hospitalization_id')['pco2_arterial'].min()
min_pco2_venous = df_PreNIPPV.groupby('hospitalization_id')['pco2_venous'].min()

# Combine min_pco2_arterial and min_pco2_venous into one dataframe
min_pco2 = pd.DataFrame({
    'pco2_arterial': min_pco2_arterial,
    'pco2_venous': min_pco2_venous
}).reset_index()

# Filter rows where min_pco2_arterial or min_pco2_venous >= 45
eligible_pco2 = min_pco2[
    (min_pco2['pco2_arterial'] >= 45) | (min_pco2['pco2_venous'] >= 45)
]['hospitalization_id']

# Filter the df to keep only eligible pco2 hospitalizations
df = df[df['hospitalization_id'].isin(eligible_pco2)].reset_index(drop=True)
admissions = df['hospitalization_id'].nunique()

#Print total admissions
print(f'Total NIPPV < 6 hrs & fio2_set <= 60 & pco2 >= 45: {len(eligible_pco2)}')
consort["total_pco2_45"] = len(eligible_pco2)

In [ ]:
# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

#Update df_PreNIPPV with only filtered admissions
df_PreNIPPV = df[df['event_time'] < df['nippv_start_time'] + pd.Timedelta(hours=1)]

# Get maximum ph_arterial and ph_venous prior to NIPPV initiation for each hospitalization_id
max_ph_arterial = df_PreNIPPV.groupby('hospitalization_id')['ph_arterial'].max()
max_ph_venous = df_PreNIPPV.groupby('hospitalization_id')['ph_venous'].max()

# Combine max_ph_arterial and max_ph_venous into one dataframe
max_ph = pd.DataFrame({
    'ph_arterial': max_ph_arterial,
    'ph_venous': max_ph_venous
}).reset_index()

# Filter to rows where max_ph_arterial or max_ph_venous <= 7.35
eligible_ph = max_ph[
    (max_ph['ph_arterial'] <= 7.35) | (max_ph['ph_venous'] <= 7.35)
]['hospitalization_id']

# Filter the df to keep only eligible ph hospitalizations
df = df[df['hospitalization_id'].isin(eligible_ph)].reset_index(drop=True)
admissions = df['hospitalization_id'].nunique()

#Print total admissions
print(f'Total NIPPV < 6 hrs & fio2_set <= 60 & pco2 >= 45 & ph <= 7.35: {len(eligible_ph)}')
consort["total_ph_7.35"] = admissions

In [ ]:
# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

# Create a new dataframe of IMV events
imv_df = df[df['device_category'] == 'IMV'].copy()
# Identify IMV hospitalizations
imv_ids = imv_df['hospitalization_id'].unique()

# Create a new dataframe of patients who died
expired_df = df[df['discharge_category'] == 'Expired'].copy()
# Identify who died
expired_ids = expired_df['hospitalization_id'].unique()

# Create separate failure flags
df['failure_imv'] = df['hospitalization_id'].isin(imv_ids).astype(int)
df['failure_death'] = df['hospitalization_id'].isin(expired_ids).astype(int)

# Overall failure: either IMV or death
df['failure'] = ((df['failure_imv'] == 1) | (df['failure_death'] == 1)).astype(int)

# Count totals (unique hospitalizations)
total_patients = df['hospitalization_id'].nunique()
total_failures = df[df['failure'] == 1]['hospitalization_id'].nunique()
imv_failures = df[df['failure_imv'] == 1]['hospitalization_id'].nunique()
death_failures = df[df['failure_death'] == 1]['hospitalization_id'].nunique()
both_failures = df[(df['failure_imv'] == 1) & (df['failure_death'] == 1)]['hospitalization_id'].nunique()

#Print total patients and failures
print(f'Total patients: {total_patients}')
consort["patients_pre_missing"] = total_patients
print(f'Total failures: {total_failures}')
consort["failures_pre_missing"] = total_failures
print(f'Total IMV failures: {imv_failures - both_failures}')
consort["imv_fail_pre_missing"] = imv_failures - both_failures
print(f'Total Death failures: {death_failures - both_failures}')
consort["death_fail_pre_missing"] = death_failures - both_failures
print(f'Both failures: {both_failures}')
consort["both_fail_pre_missing"] = both_failures

In [ ]:
# Filter events 1–12 hours after NIPPV
df_PostNIPPV_window = df[(df['time_since_nippv'] >= 1) & (df['time_since_nippv'] <= 12)]

# Sort to ensure earliest events first
df_PostNIPPV_window = df_PostNIPPV_window.sort_values(['hospitalization_id', 'event_time'])

# Get the first heart rate within the 1–12 hour window
heart_rate_window = (
    df_PostNIPPV_window.groupby('hospitalization_id')['heart_rate']
    .first()
    .reset_index()
    .rename(columns={'heart_rate': 'heart_rate_after_NIPPV'})
)

# Merge back to main df
df = df.merge(heart_rate_window, on='hospitalization_id', how='left')

In [ ]:
#Get the first respiratory rate within the 1-12 hour window
resp_rate_window = (
    df_PostNIPPV_window.groupby('hospitalization_id')['respiratory_rate']
    .first()
    .reset_index()
    .rename(columns={'respiratory_rate': 'respiratory_rate_after_NIPPV'})
)

# Merge back to main df
df = df.merge(resp_rate_window, on='hospitalization_id', how='left')

In [ ]:
# Combine arterial and venous PCO2
df_PostNIPPV_window['PostNIPPV_pco2_combined'] = df_PostNIPPV_window['pco2_arterial'].combine_first(df_PostNIPPV_window['pco2_venous'])

# Get first combined PCO2 in window
pco2_PostNIPPV_window = (
    df_PostNIPPV_window.groupby('hospitalization_id')['PostNIPPV_pco2_combined']
    .first()
    .reset_index()
    .rename(columns={'PostNIPPV_pco2_combined': 'pco2_after_NIPPV'})
)

# Merge back
df = df.merge(pco2_PostNIPPV_window, on='hospitalization_id', how='left')

In [ ]:
# Combine arterial and venous pH
df_PostNIPPV_window['PostNIPPV_ph_combined'] = df_PostNIPPV_window['ph_arterial'].combine_first(df_PostNIPPV_window['ph_venous'])

# Get first combined pH in window
ph_PostNIPPV_window = (
    df_PostNIPPV_window.groupby('hospitalization_id')['PostNIPPV_ph_combined']
    .first()
    .reset_index()
    .rename(columns={'PostNIPPV_ph_combined': 'ph_after_NIPPV'})
)

# Merge back into main df
df = df.merge(ph_PostNIPPV_window, on='hospitalization_id', how='left')


In [ ]:
#Get the first peep_set within the 1-12 hour window
peep_set_window = (
    df_PostNIPPV_window.groupby('hospitalization_id')['peep_set']
    .first()
    .reset_index()
    .rename(columns={'peep_set': 'peep_set_after_NIPPV'})
)

# Merge back to main df
df = df.merge(peep_set_window, on='hospitalization_id', how='left')

In [ ]:
#Get the first tidal_volume_obs within the 1-12 hour window
tidal_volume_obs_window = (
    df_PostNIPPV_window.groupby('hospitalization_id')['tidal_volume_obs']
    .first()
    .reset_index()
    .rename(columns={'tidal_volume_obs': 'tidal_volume_obs_after_NIPPV'})
)

# Merge back to main df
df = df.merge(tidal_volume_obs_window, on='hospitalization_id', how='left')

In [ ]:
# Get the first fio2_set within the 1–12 hour window
fio2_PostNIPPV_window = (
    df_PostNIPPV_window.groupby('hospitalization_id')['fio2_set']
    .first()
    .reset_index()
    .rename(columns={'fio2_set': 'fio2_after_NIPPV'})
)

# Merge back to main df
df = df.merge(fio2_PostNIPPV_window, on='hospitalization_id', how='left')

In [ ]:
# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

# Create combined pco2 column
df['pco2_combined'] = df['pco2_arterial'].combine_first(df['pco2_venous'])

# Sort by hospitalization and event time
df = df.sort_values(['hospitalization_id', 'event_time'])

# Get initial pco2_combined for each hospitalization
initial_pco2 = df.groupby('hospitalization_id')['pco2_combined'].first().reset_index()
initial_pco2 = initial_pco2.rename(columns={'pco2_combined': 'first_pco2'})

# Merge back into main df
df = df.merge(initial_pco2, on='hospitalization_id', how='left')

In [ ]:
# Sort by hospitalization_id and reset index after sorting
df = df.sort_values(by = ['hospitalization_id', 'event_time'])
df = df.reset_index(drop=True)

# Create combined ph column
df['ph_combined'] = df['ph_arterial'].combine_first(df['ph_venous'])

# Sort by hospitalization and event time
df = df.sort_values(['hospitalization_id', 'event_time'])

# Get initial ph_combined for each hospitalization
initial_ph = df.groupby('hospitalization_id')['ph_combined'].first().reset_index()
initial_ph = initial_ph.rename(columns={'ph_combined': 'first_ph'})

# Merge back into main df
df = df.merge(initial_ph, on='hospitalization_id', how='left')

In [ ]:
df_analytic = df.groupby('hospitalization_id').agg({
    'age_at_admission': 'first',
    'sex_category': 'first',
    'map': 'median',
    'peep_set_after_NIPPV': 'first',
    'tidal_volume_obs_after_NIPPV': 'first',
    'heart_rate_after_NIPPV': 'first',
    'respiratory_rate_after_NIPPV': 'first',
    'ph_after_NIPPV': 'first',
    'pco2_after_NIPPV': 'first',
    'fio2_after_NIPPV': 'first',
    'failure_imv':'first',
    'failure_death':'first',
    'failure': 'first'
}).reset_index()


In [ ]:
#Drop missing values
df_analytic_clean = df_analytic.dropna()

In [ ]:
#Print total number of patients before dropping missing data
admissions = df_analytic['hospitalization_id'].nunique()
print(f'Total patients before dropping missing data: {admissions}')

#Print total number of patients after dropping missing data
admissions = df_analytic_clean['hospitalization_id'].nunique()
print(f'Total patients after dropping missing data: {admissions}')

In [ ]:
# Ensure we have a copy to avoid warnings
df_analytic_clean = df_analytic_clean.copy()

# Scale continuous variables
df_analytic_clean.loc[:, 'age_scale'] = (df_analytic_clean['age_at_admission'] - df_analytic_clean['age_at_admission'].mean()) / 10
df_analytic_clean.loc[:, 'pco2_scale'] = (df_analytic_clean['pco2_after_NIPPV'] - df_analytic_clean['pco2_after_NIPPV'].mean()) / 10
df_analytic_clean.loc[:, 'ph_scale'] = (df_analytic_clean['ph_after_NIPPV'] - df_analytic_clean['ph_after_NIPPV'].mean()) / 0.1
df_analytic_clean.loc[:, 'rr_scale'] = (df_analytic_clean['respiratory_rate_after_NIPPV'] - df_analytic_clean['respiratory_rate_after_NIPPV'].mean()) / 5
df_analytic_clean.loc[:, 'hr_scale'] = (df_analytic_clean['heart_rate_after_NIPPV'] - df_analytic_clean['heart_rate_after_NIPPV'].mean()) / 10

# Binary variables
df_analytic_clean.loc[:, 'female'] = (df_analytic_clean['sex_category'] == 'Female').astype(int)

df_analytic_clean.loc[:, 'fio2_high'] = (
    df_analytic_clean['fio2_after_NIPPV'] > 0.40
).astype(int)


In [ ]:
# Export final df to CSV
df_analytic_clean.to_csv('../output/NIPPV_analytic_dataset.csv', index=False)

In [ ]:
# Total number of rows
total_rows = len(df_analytic_clean)

# Total number of failures (failure == 1)
total_failures = df_analytic_clean['failure'].sum()
imv_failures = df_analytic_clean['failure_imv'].sum()
death_failures = df_analytic_clean['failure_death'].sum()
both_failures = df_analytic_clean[(df_analytic_clean['failure_imv'] == 1) & (df_analytic_clean['failure_death'] == 1)]['hospitalization_id'].nunique()

#Print total patients and failures
print(f'Total Patients: {total_rows}')
consort["patients_post_missing"] = total_rows
print(f'Total failures: {total_failures}')
consort["failures_post_missing"] = total_failures
print(f'Total IMV failures: {imv_failures - both_failures}')
consort["imv_fail_post_missing"] = imv_failures - both_failures
print(f'Total death failures: {death_failures - both_failures}')
consort["death_fail_post_missing"] = death_failures - both_failures
print(f'Both failures: {both_failures}')
consort["both_fail_post_missing"] = both_failures

In [ ]:
# Export consort data to CSV
pd.DataFrame([consort]).to_csv('../output_to_share/consort.csv', index=False)